# Neural Network Examples - Feature Learning

This notebook aims to accomplish the following 3 goals:

1. Visually demonstrate the power of neural networks as heirarchical feature learners. 
2. Serve as a lightweight introduction to building/training neural networks in **Keras**, a user-friendly python wrapper for **tensorflow**.
3. Provide examples of text-processing neural nets including **recurrent neural networks**, as well as examples of applying **transfer learning** with pre-trained word vectors.  

In [ ]:
#Installing network structure viz

#!pip install pydot-ng
#!brew install graphviz
#!pip install pydot

In [ ]:
import pandas as pd
import seaborn as sns
import sklearn

In [ ]:
# display and plotting imports
%pylab inline 
import seaborn as sns
sns.set()
from IPython.display import SVG

import pandas as pd

# sklearn imports
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

# keras imports
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM

# gensim import for word2vec loading
from gensim.models.keyedvectors import KeyedVectors

## Digit Images Example (MNist)

We'll start by simply loading in the MNist digits data (restricted to digits 0-4), doing some PCA visualization in 2 dimensions, and seeing how well a simple linear model (softmax regression) can perform on this 2-dim representation.

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

X_digits, Y_digits = mnist.data, mnist.target.astype(np.int64)
X_digits, Y_digits = X_digits[Y_digits < 5], Y_digits[Y_digits < 5]

X_train, X_test, y_train, y_test = (train_test_split(X_digits, Y_digits, 
                                                     test_size = .2, random_state = 42))

As usual, we want to standard scale before running PCA.

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Now we'll do dimensionality reduction to 2 principal components and visualize them. We can clearly see patterns that will let us separate the various digit classes, but also a lot of messiness. **We shouldn't expect a linear model to have outstanding performance** on this representation, since the classes are clearly not linearly separable.  

In [ ]:
pca = PCA(n_components=2)

X_train_2PC = pca.fit_transform(X_train)
X_test_2PC = pca.transform(X_test)

figsize(10, 10)

sns.scatterplot(x=X_train_2PC[:, 0], y=X_train_2PC[:, 1],
                hue=y_train, alpha=.3,
                palette=sns.color_palette("colorblind", 5))
xlabel("PCA Component 1")
ylabel("PCA Component 2")
title("MNIST Digits 0-5 plotted with PCA");

Running the softmax regression confirms what we expected above - mediocre performance.

In [ ]:
lr = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs')
lr.fit(X_train_2PC, y_train)
lr.score(X_test_2PC, y_test)

### Building a Neural Network Model

We've already taken the step of standard scaling our data, so we're nearly ready to build a NN. 

We do need to adjust the format of the training labels - right now we have a 1 dimensional array of digit labels like [0, 0, 1, 3, ...], but **multi-class NN output format requires a 2-dim array with binary columns corresponding to each class** (one hot encoding). Luckily, keras provides some utilities that let us easily reformat. 

In [ ]:
y_train_cat = to_categorical(y_train)

y_train_cat

Now we get to the fun part! We'll construct our first NN with Keras. 

Here's a quick breakdown of what all of these component parts are:

 * **Sequential** : default initialization of a multi-layer network
 * **Dense** : basic hidden layer type - fully connected, meaning that for each node we learn  a weight for each of the previous layer features, just like logistic regression. The first argument is the number of nodes (output feature dimensions) 
 
 
 * **Activation** : The nonlinearity we pass through at each layer. Typical choices are 'sigmoid', 'tanh', and 'relu', 'relu' often works best. The activation at the end (softmax in this case) corresponds to the output format we want, which in this case is multi-class. We would use sigmoid for binary classification, and no activation for a regression problem.
 * **Loss** and **Optimizer** : Which loss function to optimize for, and which style of gradient descent to use. 'adam' : adaptive momentum, often works really well for optimizing.
 
 
 * **Epochs** : Number of passes through the training data. Too few can underfit, too many can  overfit. Can be optimized with validation/CV including with early stopping methods.
 * **Batch Size** : Number of samples per gradient update. CF stochastic gradient descent - we train NNs through mini-batch gradient descent, and this controlls the mini-batch size. Larger batch sizes will lead to faster epochs but run the risk of causing arrival at local minima. 

In [ ]:
'''
 In this network structure, note that we follow a very common heuristic of "funneling"
 to lower dimensional representations over time with multiple layers. Tuning the exact
 choice of number of nodes and layers is quite challenging and there aren't generically
 correct choices, but this heuristic often works pretty well.
'''

NN = Sequential()

NN.add(Dense(100,  input_dim = X_train.shape[1])) # need feature input dim (28x28) for first hidden layer
NN.add(Activation('tanh'))

NN.add(Dense(20))
NN.add(Activation('tanh'))

NN.add(Dense(10, name = '2D_layer')) # note we would typically use higher dim than this for last hidden layer
NN.add(Activation('tanh')) # naming this layer so we can extract it later

NN.add(Dense(5))
NN.add(Activation('softmax'))

NN.compile(loss='categorical_crossentropy', optimizer='adam')
NN.fit(X_train, y_train_cat, epochs=20, batch_size=512, verbose=1) # track progress as we fit


We've built and trained our model already, but even before training it we can get a summary of the network structure and visualize it to understand exactly how the model is set up.   

In [ ]:
NN.summary()

In [ ]:
SVG(model_to_dot(NN, show_shapes=True).create(prog='dot', format='svg'))

Of course we can also run predictions and score our model on the test data. It does really well!

In [ ]:
accuracy_score(y_test, NN.predict_classes(X_test))

Next, to build some visual intuition for how neural networks perform **representation learning by creating new features (often in reduced dimensions)**, we're going to do something neat: extract the two node outputs from the last hidden layer and visualize them. 

In [ ]:
feature_extractor = \
    Model(inputs=NN.input, outputs=NN.get_layer('2D_layer').output) 

X_train_NN_features_2d = feature_extractor.predict(X_train)
X_test_NN_features_2d = feature_extractor.predict(X_test)

Now we'll plot the 2 feature representation learned by the neural network, and compare with the 2 principle components of the original data as before. Look at how the **neural network has created a beautiful, linearly separable representation** of the original data. 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.scatterplot(
    x=X_train_2PC[:, 0],
    y=X_train_2PC[:, 1],
    hue=y_train,
    alpha=.1,
    palette=sns.color_palette("colorblind", 5),
    ax=axes[0],
)
axes[0].set_title("MNIST Digits 0-5 plotted with PCA")
axes[0].set_xlabel("PCA Component 1")
axes[0].set_ylabel("PCA Component 2")


sns.scatterplot(
    x=X_train_NN_features_2d[:, 0],
    y=X_train_NN_features_2d[:, 1],
    hue=y_train,
    alpha=.1,
    palette=sns.color_palette("colorblind", 5),
    ax=axes[1],
)
axes[1].set_title("MNIST Digits 0-5 plotted with Neural Network Top Layer Features: Supervised")
axes[1].set_xlabel("NN Feature 1")
axes[1].set_ylabel("NN Feature 2")

And as expected, a softmax regression shows very strong performance on the data representation that the network has learned. 

In [ ]:
lr = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs')
lr.fit(X_train_NN_features_2d, y_train)
lr.score(X_test_NN_features_2d, y_test)

Hopefully this is a visually powerful representation of the **potential predictive power to be gained from using supervised feature learning / dimension reduction techniques**. Our neural network was designed to construct a 2-dimensional, linearly separable representation of the dataset and was able to accomplish this with flying colors.

Why does this happen? Here's some intuition. The network structure is set up to terminate in a simple softmax regression mapping to the final predictions (see output layer above). So the features that are fed to that mapping must be linearly separable for the network to predict well. In this sense, the **network is designed to create a final hidden layer of linearly separable features**. The beauty of the feed-forward / back-propogation structure is that it makes it possible to algorithmically generate this representation.

This is why I like to think of neural nets as **analagous to a supervised version of PCA**. They learn features in a heirarchical fashion that ultimately represent the input data in a much simpler and more useful way for prediction. 

PCA is unsupervised so can only represent the data in a simpler way based on explained variance, but neural nets are supervised so can represent the data in a simpler way based on **target explainability**.

## Digit Exercises

In [ ]:
# EXERCISE: 
#   Reducing to a layer with 2 feature dimensions before the terminal softmax 
#   oversimplifies the model. Try adjusting the number of nodes in this layer to improve
#   the model's prediction performance.

# EXERCISE: 
#   Experiment with the network structure to try to improve performance. 
#   
#   Try adding or taking away nodes/layers. Look at the summary and visual diagram to
#   understand how the network and # of parameters are changing.

#   Try adjusting the number of epochs and the batch size. 
#   What impact do they have on performance?
#
#   Are you overfitting or underfitting? Is more or less complexity better? 
#   You can use # of parameters as a simple proxy for model complexity

# EXERCISE:
#   As you experiment with network structure, try to also incorporate dropout regularization.
#   See the bottom of the text example below for the syntax.

## Text Classification Example

Here we're going to look at a balanced binary text classification problem (sentiment detection), and train a **very simple neural network** - it will learn a 2-d representation of data just for the sake of visualization. Note that as in the digits example, this is making the model much simpler than it needs to be / below the level of complexity we would typically use in practice when building a predictive NN model.

In [ ]:
df = pd.read_csv("Data/amazon_cells_labelled.txt", sep='\t', names=['text', 'sentiment'])
# Take a look
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.sentiment, 
                                                    test_size=0.2, random_state = 42)

tfidf_vect = TfidfVectorizer(decode_error = 'ignore')
X_train = tfidf_vect.fit_transform(X_train).todense()
X_test = tfidf_vect.transform(X_test).todense()

X_train.shape

Here's our simple network to get a supervised, **2-dimensional embedding of the the tf-idf features**. 

In [ ]:
NN = Sequential()

NN.add(Dense(2, input_dim = X_train.shape[1], name = '2D_layer'))
NN.add(Activation('relu'))

NN.add(Dense(1))
NN.add(Activation('sigmoid'))

NN.compile(loss='binary_crossentropy', optimizer='adam')
NN.fit(X_train, y_train, epochs=65, batch_size=128, verbose=1)

In [ ]:
feature_extractor = \
    Model(inputs=NN.input, outputs=NN.get_layer('2D_layer').output) 

X_train_NN_features_2d = feature_extractor.predict(X_train)
X_test_NN_features_2d = feature_extractor.predict(X_test)

sns.scatterplot(x=X_train_NN_features_2d[:,0], y=X_train_NN_features_2d[:,1], hue = y_train)
plt.title('NN Learned 2D Feature Representation vs. Digit Class Label');

Neural networks can be a very powerful tool for working with text data, **provided there is enough data**. In this case, we're only training on 800 samples so we should not expect amazing generalization results from the network.

As we can see from plotting the **learned features for the test data set**, the representation that works extremely well for the training data does not generalize as well to unseen data.

In [ ]:
sns.scatterplot(x=X_test_NN_features_2d[:,0], y=X_test_NN_features_2d[:,1], hue = y_test)
plt.title('NN Learned 2D Feature Representation vs. Digit Class Label');

In [ ]:
accuracy_score(y_test, NN.predict_classes(X_test))

We can power up the complexity of this NN by adding more layers and choosing a higher number of dimensions (hidden nodes) for the top layer, but it's hard to really do much better than our simple network. This example demonstrates that there's a risk of learning a representation that's overfit to the training data. **This overfitting becomes increasingly likely if we make the network excessively complex (too many nodes + layers)**. 

In this case, **we're likely better off just using a simple model like logistic regression or naive bayes on tf-idf features due to the small data size**.

In [ ]:
lr = LogisticRegression(C = 100)
lr.fit(X_train, y_train)
print('Simple logistic score: {}'.format(lr.score(X_test, y_test)))

nb = MultinomialNB()
nb.fit(X_train, y_train)
print('Naive Bayes score: {}'.format(nb.score(X_test, y_test)))

And here is the fancy 3 layer network, which doesn't seem to be a real improvement from a simple baseline at all. 

In [ ]:
NN.summary()

In [ ]:
NN = Sequential()

NN.add(Dense(200, input_dim = X_train.shape[1]))
NN.add(Activation('relu'))
NN.add(Dropout(.3))

NN.add(Dense(100))
NN.add(Activation('relu'))
NN.add(Dropout(.3))

NN.add(Dense(50)) # 50 dimensional top-layer representation
NN.add(Activation('relu'))
NN.add(Dropout(.3))

NN.add(Dense(1))
NN.add(Activation('sigmoid'))

NN.compile(loss='binary_crossentropy', optimizer='adam')
NN.fit(X_train, y_train, epochs=30, batch_size=512, verbose=1)

In [ ]:
accuracy_score(y_test, NN.predict_classes(X_test))

## Text Classification: Moving Beyond the Simple Fully-Connected Model

When it comes to handling unstructured data, the real power of neural networks starts to become clear when we move beyond the simple computational graph of fully-connected models. With neural nets, we're able to arbitrarily piece together building blocks of matrix algebra transformations to create models that mimic the underlying patterns present in the data we're modeling. Once we've constructed a graph that mimics these patterns, we can train for the optimal weights of the linear algebraic transformations with our old friend backpropagation/gradient descent. 

For example, we can treat a text as a sequence of words and process these words in an explicitly sequential manner. This style captures the **spatial patterns** of word usage, which are extremely relevant to how humans communicate; in previous methods we've seen like word-counting/tf-idf, our models were completely unaware of order, which may cause us to lose lots of predictive signal. We'll see another example later on (convolutional neural networks) in working with images. 

Below is an example of the computational graph structure of a **Recurrent Neural Network (RNN)**. For our simple sentiment classification exercise, we can ignore the multiple output steps and treat this network as a mechanism for storing a "memory" and sequentially updating it as new information comes in word by word, then using the final state of the "memory" to make a fixed prediction.

![rnn](img/rnn3.jpg)

We'll build a modern variant of this neural net architecture, the **Long Short-Term Memory (LSTM) network**. At its core is the computational engine shown above, with some added complexity around the memory updating strategy. In the process of building the LSTM we'll also see the typical preprocessing steps required for leveraging this network architecture for NLP problems.

In [ ]:
seq_len = 100 # standardized length of each word sequence 
max_vocab = 1500 # max number of words to consider when tokenizing (based on freq)

# fit tokenizer vocab (note that it lowercases and strips punct)
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(df.text)

# standard train/val split
train_text, val_text, y_train, y_val = train_test_split(df.text, df.sentiment, 
                                                        test_size=0.2, random_state = 42)

# convert train and val texts to token sequences of standardized length 100,
# padding fills leading 0s in or cuts off sequence at 100th word
train_text = tokenizer.texts_to_sequences(train_text) 
train_text = pad_sequences(train_text, maxlen=seq_len)

val_text = tokenizer.texts_to_sequences(val_text)
val_text = pad_sequences(val_text, maxlen=seq_len)

train_text[0]

Now we use the **keras functional API** to create a computational mapping from input text sequences to output sentiment binary targets. We'll actually make the LSTM component of the model **bidirectional**, meaning that we process the text both front-to-back and back-to-front, allowing us to capture a rich set of context. 

In [ ]:
embedding_dim = 20 # hyper-parameter 

inp = Input(shape=(seq_len,)) # must specify format of input layer
x = Embedding(max_vocab, embedding_dim)(inp) # model learns its own word embeddings
x = Bidirectional(LSTM(8, recurrent_dropout=.3))(x) # bi-LSTM with regularization
y = Dense(1, activation='sigmoid')(x)

NN = Model(inp, y)
NN.summary()

The below diagram may start to confuse you more than it illuminates, but it helps emphasize the manner in which an RNN can be bidirectional:

![rnn](img/rnn.png)

Without further ado, we fit the model and track its train and validation accuracy over the training epochs. Then we'll plot the accuracy curves to get a feel for the accuracy trajectory. 

In [ ]:
NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = NN.fit(train_text, y_train, 
                 validation_data=(val_text, y_val),
                 epochs=50, batch_size=512, verbose=1)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Accuracy vs. Training Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train','Validation'])

Hmm it looks like we really can't do any better than our amazing naive bayes/logistic baseline!

In [ ]:
accuracy_score(y_val, (NN.predict(val_text)[:,0] > .5).astype(int))

## Text Classification: Leveraging The Power Of Transfer Learning

Well no, actually we can do better than that baseline. But we'll have to do more, effectively leveraging a much larger text dataset than our paltry 1000 records: we'll use **transfer learning**. Transfer learning broadly refers to the process of training neural network weights on one dataset/task, then taking those weights and applying them to a different dataset/task. It sounds like it shouldn't really work, but it turns out that since neural net weights can learn very rich representations of fairly low-level, generalizable concepts, these weights often have broad applicability. Since these weights can be learned on massive datasets and ported over to much smaller ones, this method often helps us essentially use a lot more data than we immediately have on hand for training.

One classic example of transfer learning, which we'll see here, is use of **pre-trained word vectors**. Recall that word vectors are learned via the task of predicting what words appear in similar contexts. This training task allows the vectors to capture a great deal of semantic and syntactic information that often gives relevant signal for other prediction tasks. We'll test this out by **using google's pre-trained word vectors as fixed word embeddings in our sentiment model**, instead of training embeddings on the fly. 

The code below is adapted from: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html. First we load the word vectors and build an embedding matrix.

In [ ]:
# load the w2v vectors using gensim
import gensim.downloader as api
word_vectors = api.load('glove-wiki-gigaword-50');

In [ ]:
embedding_dim = word_vectors.vector_size

In [ ]:
word_index = tokenizer.word_index


embedding_dim = word_vectors.vector_size # w2v embedding dim

# use the gensim model to build a numpy array of embeddings,
# we'll feed this array to the keras embeddings layer.
# each row i of the array will correspond to the word token assigned to that value 
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = word_vectors[word.replace("'", "")]
        embedding_matrix[i] = embedding_vector
    except: # word in our data vocab is missing in w2v, will use 0 vector for that word
        pass

Now we have what we need to define the model.

In [ ]:
inp = Input(shape=(seq_len,))
x = Embedding(len(word_index) + 1,
              embedding_dim,
              weights=[embedding_matrix], # where we feed the pretrained vecs
              trainable=False)(inp) # freeze these parameters in the model

x = Bidirectional(LSTM(64, recurrent_dropout=.1))(x)
x = Dense(32)(x) # fully connected layer on top of the output of the bi-LSTM
x = Dropout(.3)(x)
y = Dense(1, activation='sigmoid')(x)

NN = Model(inp, y)
NN.summary()

Let's train and see how we did!

In [ ]:
NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = NN.fit(train_text, y_train, 
                 validation_data=(val_text, y_val),
                 epochs=30, batch_size=512, verbose=1)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Accuracy vs. Training Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train','Validation'])

Looks like we can improve over the naive bayes baseline by at least 2% or so using the google vectors. So there's hope for deep learning after all! (once again, a big problem with this dataset is that 1000 records is just a very small size for neural net methods. It already should strike you as extremely promising that we can do so well with the transfer learning method here).